In [1]:
!pip install opencv-python  
!pip install tensorflow  
!pip install numpy  
!pip install pandas  
!pip install keras  
!pip install adam  
!pip install kwargs  
!pip install cinit  

ERROR: Could not find a version that satisfies the requirement kwargs
ERROR: No matching distribution found for kwargs


In [2]:
import sys, os  
import pandas as pd  
import numpy as np  
  
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D  
from keras.losses import categorical_crossentropy  
from keras.optimizers import Adam  
from keras.regularizers import l2  
from tensorflow.keras.utils import to_categorical

In [3]:
df=pd.read_csv(r'C:/Users/user/Downloads/fer2013.csv/fer2013.csv') 

In [4]:
X_train,train_y,X_test,test_y=[],[],[],[]  
  
for index, row in df.iterrows():  
    val=row['pixels'].split(" ")  
    try:  
        if 'Training' in row['Usage']:  
           X_train.append(np.array(val,'float32'))  
           train_y.append(row['emotion'])  
        elif 'PublicTest' in row['Usage']:  
           X_test.append(np.array(val,'float32'))  
           test_y.append(row['emotion'])  
    except:  
        print(f"error occured at index :{index} and row:{row}")  

In [5]:
num_features = 64  
num_labels = 7  
batch_size = 64  
epochs = 30  
width, height = 48, 48  

X_train = np.array(X_train,'float32')  
train_y = np.array(train_y,'float32')  
X_test = np.array(X_test,'float32')  
test_y = np.array(test_y,'float32')  

In [6]:
train_y=to_categorical(train_y, num_classes=num_labels)  
test_y=to_categorical(test_y, num_classes=num_labels)  

In [7]:
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0)  
  
X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0)  
  
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)  
  
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)  

In [8]:
model = Sequential()  
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))  
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))  

model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))  

In [9]:
model.add(Conv2D(64, (3, 3), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))  
model.add(Dropout(0.5))   

In [10]:
model.add(Conv2D(128, (3, 3), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2))) 

In [11]:
model.add(Flatten()) 
model.add(Dense(1024, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(num_labels, activation='softmax'))
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                   

In [ ]:
model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])  
model.fit(X_train, train_y,  
          batch_size=batch_size,  
          epochs=epochs,  
          verbose=1,  
          validation_data=(X_test, test_y),  
          shuffle=True)  


Epoch 1/30
449/449 [==============================] - 214s 473ms/step - loss: 1.6539 - accuracy: 0.3440 - val_loss: 1.4945 - val_accuracy: 0.4149
Epoch 2/30
449/449 [==============================] - 226s 503ms/step - loss: 1.4299 - accuracy: 0.4492 - val_loss: 1.3776 - val_accuracy: 0.4517
Epoch 3/30
449/449 [==============================] - 228s 508ms/step - loss: 1.3246 - accuracy: 0.4922 - val_loss: 1.2549 - val_accuracy: 0.5233
Epoch 4/30
449/449 [==============================] - 229s 510ms/step - loss: 1.2587 - accuracy: 0.5206 - val_loss: 1.1974 - val_accuracy: 0.5405
Epoch 5/30
 44/449 [=>............................] - ETA: 3:45 - loss: 1.2006 - accuracy: 0.5444

In [ ]:
fer_json = model.to_json()  
with open("fer.json", "w") as json_file:  
    json_file.write(fer_json)  
model.save_weights("fer.h5") 

In [ ]:
import os  
import cv2  
import numpy as np  
from keras.models import model_from_json  
from keras.preprocessing import image 

model = model_from_json(open("fer.json", "r").read())   
model.load_weights('fer.h5')  

In [ ]:
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [ ]:
cap=cv2.VideoCapture(0)  

In [ ]:
while True:  
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image  
    if not ret:  
        continue  
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)  
  
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)  
  
  
    for (x,y,w,h) in faces_detected:  
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)  
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image  
        roi_gray=cv2.resize(roi_gray,(48,48))  
        img_pixels = image.img_to_array(roi_gray)  
        img_pixels = np.expand_dims(img_pixels, axis = 0)  
        img_pixels /= 255  
  
        predictions = model.predict(img_pixels)  
  
        #find max indexed array  
        max_index = np.argmax(predictions[0])  
  
        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')  
        predicted_emotion = emotions[max_index]  
  
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)  
  
    resized_img = cv2.resize(test_img, (1000, 700))  
    cv2.imshow('Facial emotion analysis ',resized_img)  
  
  
  
    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed  
        break  
  
cap.release()  
cv2.destroyAllWindows  